In [1]:
import pandas as pd
import random

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
INPUT_FILE = '../toxicity_ratings - twitter.csv'

In [4]:
df = pd.read_csv(INPUT_FILE)

/Users/shagunjhaver/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,7,8,9,10,14,15,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df = df.dropna()

In [6]:
def get_avg_toxicity(row):
    result = (row['toxic_score'] + row['toxic_score.1'] + row['toxic_score.2'] + row['toxic_score.3'] + row['toxic_score.4'])/20.0
    return result

In [7]:
df['avg_toxic_score'] = df.apply (lambda row: get_avg_toxicity(row), axis=1)

In [8]:
df = df[['comment', 'perspective_score', 'avg_toxic_score', 'comment_ID']]

In [9]:
def get_bucket(row, field):
    if (row[field] < 0.2):
        return 0.0
    elif (row[field] < 0.4):
        return 0.2
    elif (row[field] < 0.6):
        return 0.4
    elif (row[field] < 0.8):
        return 0.6
    else:
        return 0.8

In [10]:
df['perspective_window'] = df.apply(lambda row: get_bucket(row, 'perspective_score'), axis=1)

In [11]:
df['avg_toxic_window'] = df.apply(lambda row: get_bucket(row, 'avg_toxic_score'), axis=1)


In [12]:
f = df[df['perspective_window'] != df['avg_toxic_window']]

In [13]:
len(f)

58321

In [14]:
myrange = [0.0, 0.2, 0.4, 0.6, 0.8]
sampled_dfs = []
for ow in myrange:
    g = f[f['perspective_window'] == ow]
    for iw in myrange:
        if (ow != iw):
            h = g[g['avg_toxic_window'] == iw]
            sampled_dfs.append(h.sample(1))

In [15]:
usernameDf = pd.read_csv('../usernames.csv')
usernames = set(list(usernameDf['username']))

In [16]:
final_df = pd.concat(sampled_dfs)

In [17]:
final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
final_df.head()

,comment,perspective_score,avg_toxic_score,comment_ID,perspective_window,avg_toxic_window,author
36482,"Hey @nofrillsCA, water in Guelph is high quality. We don't think you should sell #Nestle bottled water in #Guelph. @nestle makes $9 billion/year in sales from bottled water, which is a manufactured need. How much do you make from #water? What about other retailers, @LoblawsON?",0.101946,0.2,2432_18,0.0,0.2,orderly_reviewer
44840,expectations in UST Arki? Good luck! — death,0.185932,0.5,3326_14,0.0,0.4,HushedQuorum
50475,Cute that she knelt thinking it actually meant something 😂😂,0.137157,0.6,5107_15,0.0,0.6,DraftyBattery
69223,Both of them confused cause I treat em like the same hoe,0.166179,0.8,4391_23,0.0,0.8,bumpyappeasement97
16057,Yeah I’m glad they posted the maximum possible sentence for doing it so people wouldn’t copycat it anymore lol,0.229798,0.1,1525_11,0.2,0.0,favorableheads3963


In [18]:
column_to_write = ['comment', 'author', 'perspective_score', 'avg_toxic_score', 'comment_ID']

In [19]:
# final_df.to_csv('main_comments.csv', columns = column_to_write, index=False)

In [20]:
nontoxic_df = df.query('perspective_window == 0.0 & avg_toxic_window == 0.0')

In [21]:
nontoxic_sample = nontoxic_df.sample(16)

In [22]:
nontoxic_sample['author'] = nontoxic_sample.apply(lambda row: random.sample(usernames, 1)[0], axis=1)

In [23]:
nontoxic_sample.to_csv('nontoxic_comments.csv', columns = column_to_write, index=False)